0.Install Dependencies and load data

In [1]:
# !pip install pandas matplotlib scikit-learn

In [2]:
import os
import tensorflow as tf
# print(tf.config.list_physical_devices('GPU'))
import pandas as pd
import numpy as np

In [3]:
os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv')

'jigsaw-toxic-comment-classification-challenge\\train.csv\\train.csv'

In [4]:
df = pd.read_csv(os.path.join('jigsaw-toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [5]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\r\nWhy the edits made under my use...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\r\nMore\r\nI can't make any real suggestions...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \r\n\r\nThat...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \r\n\r\nUmm, theres no actual article ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\r\nAnd ... I really don't think you understa...",0,0,0,0,0,0


In [7]:
df.iloc[5]['comment_text']

'"\r\n\r\nCongratulations from me as well, use the tools well. \xa0· talk "'

In [8]:
df[df.columns[2:]].iloc[42]

toxic            1
severe_toxic     0
obscene          1
threat           0
insult           1
identity_hate    1
Name: 42, dtype: int64

1.Preprocess

In [9]:
from tensorflow.keras.layers import TextVectorization

In [10]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [11]:
MAX_FEATURES = 200000 # number of words in the vocab

In [12]:
vectorizer = TextVectorization(
    max_tokens=MAX_FEATURES, 
    output_sequence_length=1800,
    output_mode='int')

In [13]:
type(X.values)

numpy.ndarray

In [14]:
vectorizer.adapt(X.values) # teach vectorizer the vocab

In [15]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [16]:
vectorizer('Hello world! coding.')[:5]

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 288,  263, 7300,    0,    0], dtype=int64)>

In [17]:
vectorized_text = vectorizer(X.values)

In [18]:
vectorized_text

<tf.Tensor: shape=(159571, 1800), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]], dtype=int64)>

In [19]:
# MSHBAP - map, cache, shuffle, batch, prefetch
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps prevent bottlenecks

In [20]:
dataset.as_numpy_iterator().next()

(array([[   94,    13,     2, ...,     0,     0,     0],
        [   46,    65, 54014, ...,     0,     0,     0],
        [  227,     3,   147, ...,     0,     0,     0],
        ...,
        [  124,     7,     8, ...,     0,     0,     0],
        [ 2508,  1488,     4, ...,     0,     0,     0],
        [  243,    17,     6, ...,     0,     0,     0]], dtype=int64),
 array([[0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0]], dtype=int64))

In [21]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [22]:
train_generator = train.as_numpy_iterator()

In [23]:
train_generator.next()

(array([[     8,     74,    939, ...,      0,      0,      0],
        [   358,     15,     27, ...,      0,      0,      0],
        [199065,   3664,    288, ...,      0,      0,      0],
        ...,
        [     7,     19,  33630, ...,      0,      0,      0],
        [   124,      7,    124, ...,      0,      0,      0],
        [     2,   4435,     52, ...,      0,      0,      0]], dtype=int64),
 array([[1, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]], dtype=int64))

2.Build sequential model

In [24]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [25]:
# instantiate model, instantiate the sequential API then can add layers to 
model = Sequential()
# create the embedding layer, a representation of a word
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional?? LSTM layer, Wrapper for RNN
model.add(Bidirectional(LSTM(32, activation='tanh'))) # GPU acceleration is required for LSTM layer need to be 'tanh'
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# final layer
model.add(Dense(6, activation='sigmoid'))

In [26]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [28]:
# uncomment this line for training (redifine epochs to plot figure)
# history = model.fit(train, epochs=1, validation_data=val)

In [29]:
# model.save('detector_model.h5')

In [30]:
history = tf.keras.models.load_model('detector_model.h5')
history.history

In [31]:
from matplotlib import pyplot as plt

In [32]:
# Because when training the model the epochs = 1 ,so don't have anything in the chart. make it 5 or 10 will check
# plt.figure(figsize=(8,5))
# pd.DataFrame(history.history).plot()
# plt.show()

3.Make prediction

In [33]:
input_text = vectorizer('You freaking suck!')

In [34]:
input_text # before predict must pass it through as a patch

<tf.Tensor: shape=(1800,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0], dtype=int64)>

In [35]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [36]:
model.predict(np.array([input_text]))

1/1 [==============================] - 1s 677ms/step


array([[0.5005206 , 0.49734843, 0.50248134, 0.5004929 , 0.49936604,
        0.50041527]], dtype=float32)

In [37]:
res = model.predict(np.expand_dims(input_text, 0)) # same with cell above

1/1 [==============================] - 0s 70ms/step


In [38]:
batch = test.as_numpy_iterator().next()

In [39]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [40]:
batch_y

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0]], dtype=int64)

In [41]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 130ms/step


array([[1, 0, 1, 1, 1, 1],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [0, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 1],
       [1, 0, 1, 1, 0, 1]])

4.Evaluate model

In [43]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [44]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [46]:
for batch in test.as_numpy_iterator():
    # unpack the batch
    X_true, y_true = batch
    yhat = model.predict(X_true)

    y_true = y_true.flatten()
    yhat = yhat.flatten()

    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 115ms/step


In [48]:
print(f"Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}")

5.Test and Gradio

In [49]:
# !pip install gradio jinja2

In [50]:
# !pip install --upgrade ipywidgets

In [51]:
import gradio as gr

In [52]:
model = tf.keras.models.load_model('detector_model.h5')

In [53]:
input_str = vectorizer('I freaken hate you!')

In [54]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 1s 603ms/step


In [55]:
df.columns[2:]

Index(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult',
       'identity_hate'],
      dtype='object')

In [56]:
res

array([[3.0524496e-02, 9.8948803e-05, 1.6784906e-02, 7.8678865e-04,
        5.9317690e-03, 1.6741484e-03]], dtype=float32)

In [72]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)

    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    return text

In [73]:
interface = gr.Interface(fn=score_comment,
                         inputs=gr.Textbox(lines=2, placeholder='Comment to score'),
                         outputs='text')

In [74]:
interface.launch(share=False)
# interface.close()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


1/1 [==============================] - 0s 58ms/step
